In [1]:
import os
for dirname, _, filenames in os.walk('../Dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated2-rotated1-rotated1.jpg
../Dataset\archive (6)\

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [3]:
train_path= '../Dataset/archive (6)/train'
test_path='../Dataset/archive (6)/val'

In [4]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 flatten_5 (Flatten)         (None, 346112)           

In [23]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)

checkpoint = ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
callbacks = [checkpoint, lrp]

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=50,
    callbacks=callbacks
)


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 36.4518 - accuracy: 0.4500
Epoch 1: val_accuracy improved from -inf to 0.55500, saving model to model_checkpoint.h5
50/50 [==============================] - 63s 1s/step - loss: 36.4518 - accuracy: 0.4500 - val_loss: 0.6938 - val_accuracy: 0.5550 - lr: 0.0010
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.6978 - accuracy: 0.4650
Epoch 2: val_accuracy did not improve from 0.55500
50/50 [==============================] - 52s 1s/step - loss: 0.6978 - accuracy: 0.4650 - val_loss: 0.6926 - val_accuracy: 0.4900 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.6992 - accuracy: 0.4500
Epoch 3: val_accuracy did not improve from 0.55500
50/50 [==============================] - 50s 996ms/step - loss: 0.6992 - accuracy: 0.4500 - val_loss: 0.6956 - val_accuracy: 0.4450 - lr: 0.0010
Epoch 4/10
50/50 [==============================] - ETA: 0s - loss: 0.7102 - accuracy: 0.4975
Ep

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)